Importamos las librerías con las que trabajaremos

In [187]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import yellowbrick as yb
import json as js
import ast
import os
sns.set()
import datetime
import re
import emoji
import datetime
import pyarrow as pa
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings('ignore')

REVIEWS

Importamos y leemos el archivo australian_user_reviews.json, lo hacemos leyendo cada linea y haciendo un append de las mismas.

In [188]:
ruta = "australian_user_reviews.json"

filas = []

with open(ruta, encoding='MacRoman') as file:
    for line in file.readlines():
        filas.append(ast.literal_eval(line))

df_reviews = pd.DataFrame(filas)

Exploramos las columnas presentes en el archivo

In [189]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


Con el fin de evidenciar los datos hacemos un head que nos devuelva las primeras 10 filas.

In [190]:
df_reviews.head(5)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


De acuerdo a lo observado, podemos evidenciar que la columna reviews esta compues por listas de diccionarios anidados, selecciono el método explode para extraer los diccionarios de las listas

In [191]:
df_reviews2 = df_reviews.explode('reviews')

In [192]:
df_reviews2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59333 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   59333 non-null  object
 1   user_url  59333 non-null  object
 2   reviews   59305 non-null  object
dtypes: object(3)
memory usage: 1.8+ MB


Hacemos un dropna para eliminar los nan

In [193]:
df_reviews2.dropna(inplace=True)

No se pierde información relevante

In [194]:
df_reviews2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59305 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   59305 non-null  object
 1   user_url  59305 non-null  object
 2   reviews   59305 non-null  object
dtypes: object(3)
memory usage: 1.8+ MB


El método explode nos permitió eliminar las listas y consecuentemente obtener por usuario cada uno de los registros

In [195]:
df_reviews2.head(5)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."


Creamos un código con un loop para acceder a cada diccionario y extraer la data a listas para luego crear las columnas

In [196]:
funny = []
posted = []
last_edited = [] 
item_id = []
helpful = []
recommend = []
review = []

for elemento in df_reviews2['reviews']:
    for k, v in elemento.items():
        if k == 'funny':
            funny.append(v)
        if k == 'posted':
            posted.append(v)
        if k == 'last_edited':
            last_edited.append(v)
        if k == 'item_id':
            item_id.append(v)
        if k == 'helpful':
            helpful.append(v)      
        if k == 'recommend':
            recommend.append(v)
        if k == 'review':
            review.append(v)      

Creamos las nuevas columnas

In [197]:
df_reviews2['funny'] = funny
df_reviews2['posted'] = posted
df_reviews2['last_edited'] = last_edited
df_reviews2['item_id'] = item_id
df_reviews2['helpful'] = helpful
df_reviews2['recommend'] = recommend
df_reviews2['review'] = review

Eliminamos la columna reviews pues ya no es de utilidad y extrajimos toda la data que se encontraba en la misma

In [198]:
df_reviews2.drop(columns='reviews', inplace=True)

Esta es la estructura de nuestro dataframe, que mantiene el número de filas garantizando la disponibilidad de los datos

In [199]:
df_reviews2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59305 entries, 0 to 25798
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59305 non-null  object
 1   user_url     59305 non-null  object
 2   funny        59305 non-null  object
 3   posted       59305 non-null  object
 4   last_edited  59305 non-null  object
 5   item_id      59305 non-null  object
 6   helpful      59305 non-null  object
 7   recommend    59305 non-null  bool  
 8   review       59305 non-null  object
dtypes: bool(1), object(8)
memory usage: 4.1+ MB


Columna Posted:

1. Depuramos las fechas para cumplir con el formato YYYY-MM-DD: Creamos primero una columna copia de 'Posted'.
2. Eliminamos la palabra posted y el punto al final.
3. Extraemos el año.
4. Verificamos valores únicos para eliminar aquellas filas que no tienen año.


In [200]:
df_reviews2['posted_2'] = df_reviews2['posted']
df_reviews2['posted_2'] = df_reviews2['posted_2'].astype(str)
df_reviews2['posted_2'] = df_reviews2['posted_2'].str.strip('Posted ')
df_reviews2['posted_2'] = df_reviews2['posted_2'].str.strip('.')
df_reviews2["anio_posted"] = df_reviews2['posted_2'].apply(lambda x: x[-4:])
df_reviews2['anio_posted'].unique()

array(['2011', '2014', '2013', 'ry 3', '2015', '2012', 'y 20', 'y 24',
       'e 16', 'e 11', 'y 27', 't 23', 'y 16', 'h 31', 'ly 9', 'y 18',
       'y 30', 'y 28', 'ay 7', 'il 4', 'h 30', 'h 28', 'y 11', 'l 20',
       'y 26', 't 31', 'h 27', 'y 15', 't 19', 'r 14', 'ly 8', 'y 23',
       'er 9', 'y 22', 'y 29', 'ly 3', 'e 20', 'y 17', 'r 17', 't 26',
       'e 29', 'er 4', 'e 26', 'e 14', 'ch 5', 'h 19', 'l 19', 'l 18',
       'ry 7', 'l 10', 'r 15', 'l 15', 'e 22', 'y 14', 'e 27', 'ch 8',
       'ay 1', 'ch 7', 'y 10', 'r 12', 'e 25', 'h 20', 'y 13', 't 16',
       'y 31', 'ch 6', 'er 8', 'er 2', 'h 25', 'ly 7', 'e 18', 'l 22',
       'h 12', 't 13', 'ch 2', 't 15', 't 22', 'ry 9', '2010', 'h 24',
       'ay 6', 'ne 8', 'e 24', 'ne 3', 't 14', 't 28', 'l 12', 't 21',
       'ly 5', 'ry 1', 'y 12', 'e 17', 'ch 4', 'st 3', 'y 21', 'e 28',
       'ly 6', 't 29', 'ry 2', 'h 10', 'ly 4', 'er 7', 'h 11', 'r 10',
       'l 28', 'h 22', 'ry 8', 'il 2', 't 12', 'ry 4', 'y 25', 'r 16',
      

Reemplazamos columnas sin año por el valor 0 para luego proceder a eliminarlas

In [201]:
lista = []

for ind, elemento in enumerate(df_reviews2['anio_posted']):
    if elemento == '2011' or elemento == '2012' or elemento == '2013' or elemento == '2014' or elemento == '2015':
        lista.append(int(elemento))
    else:
        lista.append(0)

df_reviews2['anio_posted'] = lista

Finalmente filtramos en el dataset solo aquellas filas que tengan año en la columna posted

In [202]:
df_reviews2 = df_reviews2[df_reviews2['anio_posted'] != 0]

Cambiamos formato de fecha para ello definimos función y la aplicamos

In [203]:
def to_date(text: str):
    date_string = text
    date_format = "%B %d, %Y"
    date = datetime.datetime.strptime(date_string, date_format)
    return date

df_reviews2['posted_2'] = df_reviews2['posted_2'].apply(lambda x: to_date(x))

Reemplazamos valores en la columna posted y eliminamos la que había sido creada para el proceso

In [204]:
df_reviews2['posted'] = df_reviews2['posted_2']
df_reviews2.drop(columns='posted_2', inplace=True)

Ahora vamos a proceder con la columna helpful, en la misma haremos:

- Reemplazar el texto 'No ratings yet' por nan.
- Extraeremos los % de ayuda de los paréntesis para que al usar el algoritmo de recomendación esta columna sea de ayuda.

El código a continuación hace ambos pasos.

In [205]:
df_reviews2['helpful_2'] = df_reviews2['helpful'].str.extract(r'\((.*?)\)')
df_reviews2['helpful'] = df_reviews2['helpful_2']
df_reviews2.drop(columns='helpful_2', inplace=True)


Procederemos a depurar la columna reviews, la misma está compuesta por texto y emoticones, los mismos generan distorsión a la hora de hacer al análisis de sentimiento, por lo que vamos a reemplazar el emoticon por la palabra más adecuada para describir el sentimiento.

In [206]:
#pd.set_option('display.max_rows', None)

def emojis_count(x):
    emojis = []
    emojis_encontrados = emoji.emoji_list(x)
    if emojis_encontrados == '':
        return ''
    for e in emojis_encontrados:
        emojis.append(e['emoji'])
    return emojis
emojis = df_reviews2['review'].apply(lambda x: emojis_count(x))
emojis


0        []
0        []
0        []
1        []
1        []
         ..
25764    []
25765    []
25769    []
25771    []
25780    []
Name: review, Length: 49120, dtype: object

In [207]:
def obtener_emojis_unicos(emojis):
    return list(set(emojis))
emojis = emojis[emojis.apply(lambda x: len(x) > 0)]
emojis = emojis.apply(obtener_emojis_unicos)
emojis.drop_duplicates(inplace=True)
emojis.count()
print(emojis)

26            [™]
42            [©]
129        [©, ®]
129        [®, ™]
129        [©, ™]
129     [©, ®, ™]
2301          [®]
Name: review, dtype: object


In [208]:
emoji_reemplazos = {
    '✔': 'Approved ',
    '💯': 'Perfect ',
    '♀': 'Woman ',
    '❤': 'inLove ',
    '💋': 'Kiss ',
    '💀': 'death ',
    '™': 'Trademark ',
    '®': 'Registered Trademark ',
    '©': 'Copyright ',
    '☺': 'Smile ',
    '♥': 'Heart ',
    '👌': 'Okay ',
    '👀': 'amazed ',
    '⌛': 'long_wait ',
    '☹': 'Sad ',
    '☑': 'accomplish ',
    '😊': 'Happiness ',
    '😄': 'laugh ',
    '◀': 'recoil ',
    ':D': 'laugh ',
    ':3': 'smile',
    'xD': 'laugh_loud',
    'ô•': 'oh',
    ':c': 'adictive',
    ':)': 'happy',
    ':p': 'boring',
    '^_^': 'good',
    'D=': 'pain',
    ':(': 'sad',
    'xd': 'happiness',
    '=P': 'crazy',
    '<3': 'heart',
    ';>': 'blink',
    '(Y)': 'thumbs up',
    '*---*': 'enjoyable',
    ',3': 'amazing',
    'xt': 'unwanted',
    '-' : 'recommended',
    ':}': 'good',
    'c:': 'happy',
    ':v': 'want more',
    'x3': 'cute face',
    'XD': 'adictive',
    '(y)': 'thumbs up',
    ':P': 'bad',
    ';0': 'laugh',
    '=)': 'happy',
    'gg': 'great game',
    'GG': 'great game',
    ';D': 'great game',
    ':c': 'sad'

}

In [209]:
def replace_emojis_with_words(text):
    for emoji, replacement in emoji_reemplazos.items():
        text = text.replace(emoji, replacement)
    return text

In [210]:
df_reviews2['review'] = df_reviews2['review'].apply(replace_emojis_with_words)

In [211]:
df_reviews2['review']

0        Simple yet with great replayability. In my opi...
0                     It's unique and worth a playthrough.
0        Great atmosphere. The gunplay can be a bit chu...
1        I know what you think when you see this title ...
1        For a simple (it's actually not all that simpl...
                               ...                        
25764                                       its FUNNNNNNNN
25765    Awesome fantasy game if you don't mind the gra...
25769                                     Prettyy Mad Game
25771                                   AMAZING GAME 10/10
25780    Why I voted yes? 1. Girl characters have boobs...
Name: review, Length: 49120, dtype: object

GAMES

Importamos y leemos el archivo output_steam_games.json

In [212]:
ruta = "output_steam_games.json"

filas = []

with open(ruta, encoding='MacRoman') as file:
    for line in file.readlines():
        filas.append(js.loads((line))) # Como los tipo de valores no eran strings que es el tipo esperado por ast.litreal_eval, usamos json.loads

df_games = pd.DataFrame(filas)

In [213]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [214]:
df_games.head(10)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Vamos a extraer los datos de genres pues  specs y tags no es necesario desanidarlo

In [215]:
df_games = df_games.explode('genres')

In [216]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 163147 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     59243 non-null  object
 1   genres        71554 non-null  object
 2   app_name      74834 non-null  object
 3   title         72786 non-null  object
 4   url           74837 non-null  object
 5   release_date  72735 non-null  object
 6   tags          74652 non-null  object
 7   reviews_url   74834 non-null  object
 8   specs         73896 non-null  object
 9   price         71491 non-null  object
 10  early_access  74837 non-null  object
 11  id            74834 non-null  object
 12  developer     71358 non-null  object
dtypes: object(13)
memory usage: 17.4+ MB


eliminamos los nan del archivo

In [217]:
df_games.dropna(subset=['genres','publisher','app_name', 'price','release_date','id'], inplace=True)
df_games.info()


<class 'pandas.core.frame.DataFrame'>
Index: 56276 entries, 88310 to 120443
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     56276 non-null  object
 1   genres        56276 non-null  object
 2   app_name      56276 non-null  object
 3   title         56276 non-null  object
 4   url           56276 non-null  object
 5   release_date  56276 non-null  object
 6   tags          56260 non-null  object
 7   reviews_url   56276 non-null  object
 8   specs         55739 non-null  object
 9   price         56276 non-null  object
 10  early_access  56276 non-null  object
 11  id            56276 non-null  object
 12  developer     56163 non-null  object
dtypes: object(13)
memory usage: 6.0+ MB


Vamos a separar la columna release date y crear una para el año ya que se identificaros valores en formato que no son fecha y palabras.

In [218]:
df_games['release_date_anio'] = df_games['release_date'] .str.split('-').str[0]
df_games['release_date_anio_2'] = df_games['release_date_anio'].str[-4:]
df_games['release_date_anio_2'] = df_games['release_date_anio_2'].replace('OON™', '')
df_games['release_date_anio_2'] = df_games['release_date_anio_2'].replace('SOON', '')
df_games['release_date_anio'] = df_games['release_date_anio_2']


In [219]:
a = df_games['release_date_anio'].value_counts()
a

release_date_anio
2017    19968
2016    13793
2015     9157
2014     4688
2013     2557
2012     2005
2011      933
2010      652
2009      478
2008      319
2007      248
2006      213
2018      209
2005      123
2003      117
2001       99
2004       91
1998       74
1999       73
1997       63
2002       61
2000       55
1996       55
1994       51
1995       48
1993       39
1992       24
1991       19
1989       11
1990       11
1988        9
2019        6
1984        5
1987        5
1986        5
            5
1983        3
2021        3
1985        1
Name: count, dtype: int64

In [220]:
df_games.drop(columns=['release_date_anio_2'], inplace = True)

Cambiamos el tipo de dato de price que se encuentra como objeto, primero haciendo un replace de las filas free to play en todas sus formas también hay juegos que no tienen precio, estos también los imputaremos con 0.

In [221]:
df_games['price'] = df_games['price'].replace(['free to play', 'freetoplay', 'Install Now', 'Third-party', 'Free HITMAN™ Holiday Pack','Play the Demo','Free to Play','Free To Play','Free','Play for Free!','Free to Try','Free to Use','Free Demo','Play Now','Free Mod','Play WARMACHINE: Tactics Demo' ], 0)

In [222]:
a = df_games['price'].value_counts()
a


price
9.99      7658
4.99      7642
0.99      5349
1.99      4851
2.99      4785
          ... 
99.00        1
771.71       1
27.99        1
4.29         1
202.76       1
Name: count, Length: 128, dtype: int64

In [223]:
df_games['price'] = df_games['price'].astype(float)

In [224]:
df_games.head(5)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,release_date_anio
88310,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018
88310,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018
88310,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018
88310,Kotoshiro,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018
88310,Kotoshiro,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018


ITEMS

Importamos y leemos el archivo australian_users_items.json

In [225]:
ruta = "australian_users_items.json"

filas = []

with open(ruta, encoding='MacRoman') as file:
    for line in file.readlines():
        filas.append(ast.literal_eval(line))

df_user_items = pd.DataFrame(filas)

In [226]:
df_user_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [227]:
df_user_items.head(10)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
5,MinxIsBetterThanPotatoes,371,76561198004744620,http://steamcommunity.com/id/MinxIsBetterThanP...,"[{'item_id': '50', 'item_name': 'Half-Life: Op..."
6,NitemarePK,304,76561197990951820,http://steamcommunity.com/id/NitemarePK,"[{'item_id': '240', 'item_name': 'Counter-Stri..."
7,themanwich,258,76561198035296505,http://steamcommunity.com/id/themanwich,"[{'item_id': '220', 'item_name': 'Half-Life 2'..."
8,maplemage,629,76561198026584251,http://steamcommunity.com/id/maplemage,"[{'item_id': '240', 'item_name': 'Counter-Stri..."
9,Wackky,0,76561198039117046,http://steamcommunity.com/id/Wackky,[]


Como podemos observar ítems esta anidado en listas de diccionarios

In [228]:
df_user_items['items'].keys

<bound method Series.keys of 0        [{'item_id': '10', 'item_name': 'Counter-Strik...
1        [{'item_id': '10', 'item_name': 'Counter-Strik...
2        [{'item_id': '1200', 'item_name': 'Red Orchest...
3        [{'item_id': '10', 'item_name': 'Counter-Strik...
4        [{'item_id': '300', 'item_name': 'Day of Defea...
                               ...                        
88305    [{'item_id': '413850', 'item_name': 'CS:GO Pla...
88306    [{'item_id': '11020', 'item_name': 'TrackMania...
88307                                                   []
88308    [{'item_id': '304930', 'item_name': 'Unturned'...
88309                                                   []
Name: items, Length: 88310, dtype: object>

In [229]:
lista = df_user_items['items']
lista_items_id = []
lista_items_name = []

Hacemos un explode con el fin de acceder a las listas de la columna ítems y dejar únicamente los diccionarios

In [230]:
df_user_items2 = df_user_items.explode('items')

In [231]:
df_user_items2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5170015 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   user_id      object
 1   items_count  int64 
 2   steam_id     object
 3   user_url     object
 4   items        object
dtypes: int64(1), object(4)
memory usage: 236.7+ MB


Luego del explode y tener los diccionarios listados por usuario, procedemos a hacer un drop na para eliminar los diccionarios vacíos que no son de utilidad

In [232]:
df_user_items2.dropna(inplace=True)

In [233]:
df_user_items2.head(5)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike..."
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ..."
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."


Creamos un dataframe de prueba para hacer el proceso de extracción de datos del diccionario

In [234]:
dict = df_user_items2['items']
user_id = []
listaindex = []
listadict = []

for k, v in dict.items():
    listadict.append(v)      

In [235]:
dfprueba = pd.DataFrame()
dfprueba['user_id'] = df_user_items2['user_id']
dfprueba['dict'] = listadict

In [236]:
dfprueba.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5153209 entries, 0 to 88308
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  object
 1   dict     object
dtypes: object(2)
memory usage: 117.9+ MB


In [237]:
dfprueba.head(5)

,user_id,dict
0,76561197970982479,"{'item_id': '10', 'item_name': 'Counter-Strike..."
0,76561197970982479,"{'item_id': '20', 'item_name': 'Team Fortress ..."
0,76561197970982479,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
0,76561197970982479,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
0,76561197970982479,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."


Recorremos cada diccionario y creamos listas para guardar los datos

In [238]:
item_id2 = []
item_name = []
playtime_forever = [] 
playtime_2weeks = []

for elemento in dfprueba['dict']:
    for k, v in elemento.items():
        if k == 'item_id':
            item_id2.append(v)
        if k == 'item_name':
            item_name.append(v)
        if k == 'playtime_forever':
            playtime_forever.append(v)
        if k == 'playtime_2weeks':
            playtime_2weeks.append(v)

Ingestamos los datos en nuevas columnas 

In [239]:
df_user_items2['item_id'] = item_id2
df_user_items2['item_name'] = item_name
df_user_items2['playtime_forever'] = playtime_forever
df_user_items2['playtime_2weeks'] = playtime_2weeks

In [240]:
df_user_items2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5153209 entries, 0 to 88308
Data columns (total 9 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   items_count       int64 
 2   steam_id          object
 3   user_url          object
 4   items             object
 5   item_id           object
 6   item_name         object
 7   playtime_forever  int64 
 8   playtime_2weeks   int64 
dtypes: int64(3), object(6)
memory usage: 393.2+ MB


Eliminamos la columna con diccionarios pues ya extrajimos la data

In [241]:
df_user_items2.drop(columns='items', inplace=True)

In [242]:
df_user_items2.head(5)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0,0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7,0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0,0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0,0


Reemplazamos de acuerdo al nombre del título, buscando estandarizar y tener valores únicos

In [243]:
df_user_items2['item_name'].replace(to_replace='Counter-Strike', value='Counter-Strike', inplace=True)
df_user_items2['item_name'].replace(to_replace='COUNter-strike', value='Counter-Strike', inplace=True)
df_user_items2['item_name'].replace(to_replace='COUNter-strike', value='Counter-Strike', inplace=True)
df_user_items2['item_name'].replace(to_replace='half-life', value='Half-life', inplace=True)

Revisamos que no haya valores atípicos en los nombres de los juegos

In [244]:
resultado = df_user_items2['item_name'].value_counts().sort_values()
print(resultado)

item_name
Arachnophobia                           1
Pixel: ru¬≤                             1
Orbital Strike: Arena                   1
Trigonarium                             1
Typing Instructor Platinum 21           1
                                    ...  
Left 4 Dead 2                       37044
Unturned                            38682
Garry's Mod                         43301
Counter-Strike: Global Offensive    43776
Dota 2 Test                         49571
Name: count, Length: 10947, dtype: int64


Eliminamos las columnas steam id, user url y playtime 2 weeks pues no nos serán útiles para nuestro trabajo

In [245]:
df_user_items2 = df_user_items2.drop(columns=['steam_id','item_name', 'playtime_2weeks'])


In [246]:
df_user_items2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5153209 entries, 0 to 88308
Data columns (total 5 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   items_count       int64 
 2   user_url          object
 3   item_id           object
 4   playtime_forever  int64 
dtypes: int64(2), object(3)
memory usage: 235.9+ MB


Buscando optimizar las funciones, para la funcion genre, depuraremos el archivo items dejando unicamente las columnas que toma la funcion

In [260]:
df_items_genre = df_user_items2[['item_id', 'playtime_forever']]
df_items_genre.to_csv('df_items_genre.csv', index=False)

In [261]:
df_items_userforgenre = df_user_items2[['item_id', 'playtime_forever','user_id']]
df_items_userforgenre.to_csv('df_items_userforgenre.csv', index=False)

In [262]:
df_games_userforgente = df_games[['id', 'genres']]
df_games_userforgente.to_csv('df_games_userforgente.csv', index=False)

Buscando optimizar las funciones, para la user_id, depuraremos el archivo items dejando unicamente las columnas que toma la funcion

In [263]:
df_items_userid = df_user_items2[['items_count', 'item_id','user_id']]
df_items_userid.to_csv('df_items_userid.csv', index=False)

In [268]:
df_games_userid = df_games[['id','price']]
df_games_userid.to_csv('df_games_userid.csv', index=False)


In [269]:
df_reviews_userid = df_reviews2[['user_id','recommend']]
df_reviews_userid.to_csv('df_reviews_userid.csv', index=False)

Depuración archivo para función developer

In [270]:
df_games_developer = df_games[['price','developer','id','release_date_anio']]
df_games_developer.to_csv('df_games_developer.csv', index=False)

Ahora vamos a depurar y generar arhivo para la funcion countreviews

In [271]:
df_reviews_countreviews = df_reviews2.copy()
df_reviews_countreviews = df_reviews_countreviews[['posted','user_id','recommend']]
df_reviews_countreviews.to_csv('df_reviews_countreviews.csv', index=False)


Ahora vamos a depurar y generar arhivo para la funcion genres con el dataframe games

In [272]:
df_games_genre = df_games.copy()
df_games_genre = df_games_genre[['id','genres']]
df_games_genre.to_csv('df_games_genre.csv', index=False)

Vamos ahora a depurar el archivo games para la funcion de sentiment analysis

In [273]:
df_games_sentiment = df_games.copy()
df_games_sentiment = df_games_sentiment[['id','release_date_anio']]
df_games_sentiment.to_csv('df_games_sentiment.csv', index=False)

Archivo para función genre depurado:

In [275]:
df_items_userforgenre = pd.read_csv(r'df_items_userforgenre.csv')
df_items_userforgenre = df_items_userforgenre.sample(frac=0.2, random_state=42)
df_items_userforgenre.to_csv('df_items_userforgenre.csv')

In [284]:
games_gen = pd.read_csv(r'df_games_genre.csv')
items = pd.read_csv(r'df_items_genre.csv')
games_gen.drop_duplicates(subset=['id'], inplace=True) # Elimina los juegos repetidos, para al hacer match entre ambos datasets no duplicar
df_game_time = items[['item_id','playtime_forever']] # Solo trae estas dos columnas que son las necesarias para el codigo.
df_agrupado = df_game_time.groupby('item_id')['playtime_forever'].sum() # agrupa por juego el total jugado.
merged = pd.merge(games_gen, df_agrupado, left_on='id', right_on='item_id', how='left') # Hago el merge
merged.dropna(subset=['playtime_forever'], inplace=True)
merged.to_csv('merged_genres.csv', index=False)

Creamos un archivo CSV con cada uno de los dataframe trabajados y ya depurados

In [283]:
df_reviews_userid.to_csv('df_reviews_userid.csv', index=False)
df_games_userforgente.to_csv('df_games_userforgente.csv', index=False)
df_games_userid.to_csv('df_games_userid.csv', index=False)
df_reviews_countreviews.to_csv('df_reviews_countreviews.csv', index=False)
df_games_developer.to_csv("df_games_developer.csv", index = False)
df_games_sentiment.to_csv('df_games_sentiment.csv', index=False)
df_reviews2.to_csv('reviews.csv', index = False)
df_games.to_csv('games.csv', index=False)
df_reviews2.to_csv('reviews.csv', index=False)


Ahora hago lo mismo para la función user_data

In [281]:
df_items_user_sample = pd.read_csv(r'df_items_userid.csv')
df_items_user_sample = df_items_user_sample.sample(frac=0.10, random_state=42)
df_items_user_sample.to_csv('df_items_user_sample.csv')

Para poder ejecutar el modelo de recomendacion, insertamos un valor vacio en columnas con na, seleccionamos unicamente los generos con mas relevancia en el dataframe y  hacemos una muestra:

In [282]:
df_modelo_recomendacion = pd.read_csv(r'games.csv')
df_modelo_recomendacion = df_modelo_recomendacion[['publisher', 'genres', 'id', 'app_name']]
df_modelo_recomendacion.dropna(inplace=True)
df_modelo_recomendacion= df_modelo_recomendacion[df_modelo_recomendacion['genres'] != 'Free to Play']
df_modelo_recomendacion= df_modelo_recomendacion[df_modelo_recomendacion['genres'] != 'Early Access']
df_modelo_recomendacion= df_modelo_recomendacion[df_modelo_recomendacion['genres'] != 'Sports']
df_modelo_recomendacion= df_modelo_recomendacion[df_modelo_recomendacion['genres'] != 'Racing']
df_modelo_recomendacion= df_modelo_recomendacion[df_modelo_recomendacion['genres'] != 'Massively Multiplayer']
df_modelo_recomendacion= df_modelo_recomendacion[df_modelo_recomendacion['genres'] != 'Design &amp; Illustration']
df_modelo_recomendacion= df_modelo_recomendacion[df_modelo_recomendacion['genres'] != 'Utilities']
df_modelo_recomendacion= df_modelo_recomendacion[df_modelo_recomendacion['genres'] != 'Web Publishing']
df_modelo_recomendacion= df_modelo_recomendacion[df_modelo_recomendacion['genres'] != 'Animation &amp; Modeling']
df_modelo_recomendacion= df_modelo_recomendacion[df_modelo_recomendacion['genres'] != 'Education']
df_modelo_recomendacion= df_modelo_recomendacion[df_modelo_recomendacion['genres'] != 'Software Training']
df_modelo_recomendacion= df_modelo_recomendacion[df_modelo_recomendacion['genres'] != 'Audio Production']
df_modelo_recomendacion= df_modelo_recomendacion[df_modelo_recomendacion['genres'] != 'Video Production']
df_modelo_recomendacion= df_modelo_recomendacion[df_modelo_recomendacion['genres'] != 'Photo Editing']
df_modelo_recomendacion= df_modelo_recomendacion[df_modelo_recomendacion['genres'] != 'Accounting']
df_modelo_recomendacion = df_modelo_recomendacion.sample(frac=0.12, random_state=42)
df_modelo_recomendacion.to_csv('df_modelo_recomendacion.csv', index=False)